# 심층 신경망

## 2개의 층

In [1]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

* 은닉층(Hidden layer) : 입력층과 출력층 사이에 있는 모든 층
* 은닉층에는 활성화 함수 존재
    * 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수
    * 은닉층의 활성화 함수는 비교적 자유롭다

* 신경망의 출력층 (분류 문제) : 시그모이드 함수 or 소프트맥스 함수
    * 이진 분류 : 시그모이드 함수
    * 다중 분류 : 소프트맥스 함수

* 회귀 신경망의 출력층 : 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다
    * 출력층의 선형 방정식의 계산을 그대로 출력
* 이렇게 하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않는다

In [3]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

* 위의 코드를 보면, 출력층 이전의 뉴런의 갯수를 보면 10개보다 많다
* 만약 은닉층의 뉴런이 10개보다 적다면 부족한 정보가 전달될 것 

---

## 심층 신경망 만들기

* Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 아래와 같이 dense1과 dense2를 리스트 항목으로 전달 

In [4]:
model_1 = keras.Sequential([dense1, dense2])

In [5]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


* 층의 이름을 지정하지 않으면, dense라고 이름을 자동 생성 
* 출력 크기 : None, 100 
    * 첫 번째 차원 : 샘플의 개수
    * 샘플의 개수가 아직 정의되어 있지 않기 때문에 None 

* fit() 메서드 : 훈련 데이터를 주입하면, 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번 걸쳐 **미니배치 경사 하강법** 단계 수행 
* 케라스의 기본 미니배치 크기 : 32
    * fit() 메서드에서 batch_size 매개변수로 변경 가능 

---

### 층을 추가하는 다른 방법
* Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체 생성

In [6]:
model_2 = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden1'),
    keras.layers.Dense(100, activation='sigmoid',  name='hidden2'),
    keras.layers.Dense(100, activation='relu',  name='hidden3'),
    keras.layers.Dense(10, activation='softmax', name='output')
])

In [7]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 100)               78500     
                                                                 
 hidden2 (Dense)             (None, 100)               10100     
                                                                 
 hidden3 (Dense)             (None, 100)               10100     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________


* Sequential 클래스의 객체를 만들고, 이 객체의 add() 메서드를 호출하여 층을 추가하는 방법

In [8]:
model_3 = keras.Sequential()
model_3.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model_3.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
model_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
model_3.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model_3.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5632 - accuracy: 0.8095
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4076 - accuracy: 0.8541
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3738 - accuracy: 0.8652
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3505 - accuracy: 0.8732
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3331 - accuracy: 0.8797


---

## 렐루 활성화 함수 (ReLU)
* 렐루 함수 : 이미지 분류 모델의 은닉층에 많이 사용하는 활성화 함수 
* 시그모이드 함수는 층이 많을수록 활성화 함수의 양쪽 끝에서 변화가 작기 때문에 학습이 어려워진다
    * 렐루함수는 이런 문제가 없으며 계산도 간단

* Flatten 클래스 : 배치 차원을 제외하고, 나머지 입력 차원을 모두 일렬로 펼치는 역할만 수행
    * 곱해지는 가중치나 절편 X 
* Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가
    * Flatten 층은 다음 코드처럼 입력층 바로 뒤에 추가

In [11]:
model_relu = keras.Sequential()
model_relu.add(keras.layers.Flatten(input_shape=(28, 28)))
model_relu.add(keras.layers.Dense(100, activation='relu'))
model_relu.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
model_relu.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 데이터 전처리 다시 수행

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [14]:
model_relu.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model_relu.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5282 - accuracy: 0.8134
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3888 - accuracy: 0.8618
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3527 - accuracy: 0.8721
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3319 - accuracy: 0.8808
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3177 - accuracy: 0.8871


In [15]:
model_relu.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3794 - accuracy: 0.8730


[0.37936070561408997, 0.8730000257492065]

---

### 옵티마이저
* 케라스는 기본적으로 미니배치 경사 하강법을 사용하며 미니배치 개수는 32개 
* fit() 메서드의 batch_size 매개변수에서 이를 조정 가능 
* 케라스의 기본 경사 하강법 알고리즘인 **RMSprop** 사용
* 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공. 이들을 옵티마이저(Optimizer)라고 한다 

In [16]:
model_relu.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [17]:
sgd = keras.optimizers.SGD()
model_relu.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

* 옵티마이저의 SGD 클래스를 사용하면 학습률등, 파라미터 값을 변경 가능

In [18]:
sgd = keras.optimizers.SGD(learning_rate=0.01)

* 기본 경사 하강법 옵티마이저 : 모두 SGD 클래스에서 제공
    * SGD 클래스의 momentum 매개변수의 기본값 : 0
    * 0보다 큰 값으로 지정 : 마치 이전의 그레디언트를 가속도처럼 사용하는 모멘텀 최적화(momentum optimization) 사용
* 보통 momentum 매개변수는 0.9 이상을 지정
* SGD 클래스의 nesterov 매개변수를 기본값으로 False에서 True로 변경 : 네스테로프 모멘텀 최적화(네스테로프 가속 경사) 사용
* 네스테로프 모멘텀 : 모멘텀 최적화를 2번 반복하여 구현 
    * 대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공 

In [19]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

* 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다
    * 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다
    * 이런 학습률을 **적응적 학습률(adaptive learning rate)** 이라고 한다
    * 이런 방식들은 학습률 매개변수를 튜닝하는 수고를 덜 수 있는 것이 장점

* 적응적 학습률을 사용하는 대표적인 옵티마이저 : Adagrad,  RMSprop 
* 각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정
    * 기본값으로 rmsprop이 설정

In [20]:
adagrad = keras.optimizers.Adagrad(learning_rate=0.01)

rmsprop = keras.optimizers.RMSprop()

In [21]:
model_relu = keras.Sequential()
model_relu.add(keras.layers.Flatten(input_shape=(28, 28)))
model_relu.add(keras.layers.Dense(100, activation='relu'))
model_relu.add(keras.layers.Dense(10, activation='softmax'))

In [22]:
model_relu.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')
model_relu.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5316 - accuracy: 0.8125
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3948 - accuracy: 0.8587
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3566 - accuracy: 0.8717
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3326 - accuracy: 0.8808
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3198 - accuracy: 0.8865


In [23]:
model_relu.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3787 - accuracy: 0.8734


[0.3787129819393158, 0.8734166622161865]